### 개념정리
- single vs Multi
- GPU VS Node(computer 1대)
- Single Node Multi GPU : 한대의 컴퓨터에 여러 gpu가 설치됨
- Multi Node  Mulit GPU : 여러 컴퓨터 여러 GPU

### Model parallel
- 모델을 다중 GPU에 분산하여 학습하는것
- 모델 병렬화
- alexnet
- 병목, 파이프라인 등의 어려움으로 고난이도임
- .to(gpu) 해서 보내줌

### Data Parallel
- 데이터를 다중 GPU 에 분산하여 학습
- 각각의 LOSS값을 통해 역전파를 한뒤 gradient의 평균을 구하여 weight update를 해줌
- loss값 도출과 grad평균으로 weight update는 하나의 gpu에서 함
- mini batch 수식과 유사하나 한번에 여러 gpu에서 수행함

    #### 두가지 방식
    - DataParallel
        - 단순히 데이터 분배 후 평균
        - GPU 사용 불균형 문제 발생
        - 중앙 coordinate GPU에 병목현상 발생        
        - __이를 고려하여 batch size 를 감소시켜줘야함__
        - GIL 문제
    - DistributedDataParallel
        - 각각 cpu를 할당하여 loss, grad를 구한뒤 평균을 구함
        - 하나의 gpu가 coordinate하지 않음
         

DataParallel

In [ ]:
paralle_model = torch.nn.DataParallel(model)


DistributedDataParallel
- sampler를 만들어줘야함
- shuffle = False
- pin_memory = True
- num_workers : gpu의 갯수 곱하기 4 해준다

In [ ]:
train_sampler = torch.utils.data.distributed.DistributedSampler(train_data)
shuffle = False
# 메모리에 바로바로 올릴 수 있도록 절차를 간소하게 하는 것.
pin_memory = True

trainloader = torch.utils.data.DataLoader(train_data, batch_size = 20, shuffle = False, 
            pin_memory = pin_memory, num_workers = 4, 
            shuffle = shuffle, sampler = train_sampler)
# 알아서 gpu에 각각 데이터를 넣어줌

main 함수

In [ ]:
def main():
    n_gpus = torch.cuda.device_count()
    torch.multiprocessing.spawn(main_worker, nprocs = n_gpus, args = (n_gpus, ))
    # n_gpus : gpu수 * 4
def main_worker(gpu, n_gpus):

    image_size = 224
    batch_size = 512
    num_worker = 8
    epochs = ...

    batch_size = int(batch_size / n_gpus)
    num_worker = int(num_worker / n_gpus)

    torch.distributed.init_process_group(
                                            # gpu 끼리 통신하는 멀티프로세싱 통신 규약 정의
            backend = 'nccl', init_metod = 'tcp://127.0.0.1:2569 ', world_zier = n_gpus, rank = gpu)
    )

    model = MODEL

    torch.cuda.set_device(gpu)
    model = model.cuda(gpu)
    model = torch.nn.parallel.DistributedDataParallel(model, divice_ids=[gpu])
